In [17]:
import json
import numpy as np
import sys
sys.path.append('../src')
import cpu

with open('C:/Users/David/Documents/Coding/daveNES/tests/ProcessorTests-main/nes6502/v1/e9.json') as f:
    test = json.load(f)

`! pytest --html="report_%date:/=-%_%time::=-%.html"`

In [25]:
subtest = 100

In [26]:
test[0]

{'name': 'e9 20 ae',
 'initial': {'pc': 60024,
  's': 171,
  'a': 50,
  'x': 168,
  'y': 179,
  'p': 106,
  'ram': [[60024, 233], [60025, 32], [60026, 174]]},
 'final': {'pc': 60026,
  's': 171,
  'a': 17,
  'x': 168,
  'y': 179,
  'p': 41,
  'ram': [[60024, 233], [60025, 32], [60026, 174]]},
 'cycles': [[60024, 233, 'read'], [60025, 32, 'read']]}

In [27]:
test[subtest]['initial']['ram']

[[26488, 233], [26489, 155], [26490, 160]]

In [28]:
def init_daveNES(test):
    daveNES = cpu.MOS6502(debug = False)
    daveNES.initialise_RAM()

    # Load test program
    for val in test['initial']['ram']:
        print(f'{val[0]}, {val[1]}')
        daveNES.ram.memory[val[0]] = np.uint8(val[1])
    daveNES.r_program_counter = test['initial']['pc']
    daveNES.r_stack_pointer = test['initial']['s']
    daveNES.r_accumulator = test['initial']['a']
    daveNES.r_index_X = test['initial']['x']
    daveNES.r_index_Y = test['initial']['y']
    daveNES.value_to_status(test['initial']['p'])

    return daveNES

In [29]:
daveNES = init_daveNES(test[subtest])

26488, 233
26489, 155
26490, 160


In [30]:
daveNES.step_program()

0xe9, SBC
PC: 0x6778, SP: 0xd2, A: 0xba, X: 0xa3, Y: 0x68, [1, 1, 1, 0, 1, 1, 0, 1]
SBC performed


In [31]:
print(f'{test[subtest]["final"]["pc"]} : {daveNES.r_program_counter}')
print(f'{test[subtest]["final"]["s"]} : {daveNES.r_stack_pointer}')
print(f'{test[subtest]["final"]["a"]} : {daveNES.r_accumulator}')
print(f'{test[subtest]["final"]["x"]} : {daveNES.r_index_X}')
print(f'{test[subtest]["final"]["y"]} : {daveNES.r_index_Y}')
print(f'{test[subtest]["final"]["p"]:08b} : {[int(daveNES.r_status[k]) for k in daveNES.r_status.keys()][::-1]}')

26490 : 26490
210 : 210
31 : 31
163 : 163
104 : 104
00101101 : [0, 1, 1, 0, 1, 1, 0, 1]
